In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import textwrap
import os
from google.colab import drive
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import cmudict
import string
import re
from nltk.corpus import stopwords

In [ ]:
url='https://docs.google.com/spreadsheets/d/1J0vxdeTLfj_I1OWui6V-Nwp1XF989IOB/edit?usp=drive_link&ouid=112152660510254781901&rtpof=true&sd=true'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
url_df = pd.read_excel(url)
url_df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
url_df = url_df[['URL_ID', 'URL']]
url_df.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [ ]:
# function to extract article from an url
def extract_article(url):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content of the page using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the title of the article
            title = soup.find('title').text

            # Find the main content of the article (assuming it's within a <div> with a specific class)
            article_div = soup.find('div', class_='td-post-content tagdiv-type')
            article_div2 = soup.find('div', class_='td_block_wrap tdb_single_content tdi_130 td-pb-border-top td_block_template_1 td-post-content tagdiv-type')

            if article_div:
                # Extract the text from the article
                article_text = article_div.get_text()

                # Clean up extra whitespace and newlines
                article_text = ' '.join(article_text.split())

                return {
                    'title': title,
                    'text': article_text
                }
            elif article_div2:
                # Extract the text from the article
                article_text = article_div2.get_text()

                # Clean up extra whitespace and newlines
                article_text = ' '.join(article_text.split())

                return {
                    'title': title,
                    'text': article_text
                }
            else:
                return {'error': 'No article content found.'}
        else:
            return {'error': f'Failed to fetch the page. Status code: {response.status_code}'}
    except Exception as e:
        return {'error': str(e)}



In [ ]:
# Example usage:
# Replace with the URL of the article you want to extract
url = 'https://insights.blackcoffer.com/rise-of-e-health-and-its-imapct-on-humans-by-the-year-2030-2/'
article_data = extract_article(url)

if 'error' in article_data:
    print(f"Error: {article_data['error']}")
else:
    print(f"Title: {article_data['title']}")
    wrapped_text = '\n'.join(textwrap.wrap(article_data['text'], width=150))
    print(f"Text: {wrapped_text}")

Title: Rise of e-health and its impact on humans by the year 2030 | Blackcoffer Insights
Text: 2020 was the year the world was ravaged by the SarsCov2 virus. This notorious virus brought about a pandemic that would go on to change the course of
humanity. From that point forth daily lives of everyone across the world changed. With widespread stringent lockdowns, the entire world came to a
sharp halt. Not only was the general populace affected, but the pandemic also affected all industries. The pandemic did not even spare critical
industries, like healthcare and security. While these industries were required to function for the benefit of society, their daily operations changed
drastically. But just as human nature prevails, we rose from this adversity. Post pandemic era saw the rise of new technologies that could aid
overcome the restrictions put forth by the pandemic. In this article, we will specifically focus on the healthcare industries, innovations done in the
industry, and the imp

In [ ]:
url_df['content'] = url_df['URL'].apply(extract_article)

In [ ]:
url_df.head()

,URL_ID,URL,content
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,{'title': 'Rise of e-health and its impact on ...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,{'title': 'Rise of e-health and its impact on ...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...


In [ ]:
# function to concate title and text in a dict
def concate(dict1):
  if 'error' in  dict1:
     return f"Error: {dict1['error']}"
  else:
    return dict1['title'] + '. ' + dict1['text']


In [ ]:
url_df['text'] = url_df['content'].apply(concate)

In [ ]:
url_df[url_df['text'].str.contains('Error')]

,URL_ID,URL,content,text
24,11668.0,https://insights.blackcoffer.com/how-neural-ne...,{'error': 'Failed to fetch the page. Status co...,Error: Failed to fetch the page. Status code: 404
29,13977.0,https://insights.blackcoffer.com/ai-human-robo...,{'title': 'How Robo Human will Impact the Futu...,How Robo Human will Impact the Future? | Black...
37,17671.4,https://insights.blackcoffer.com/covid-19-envi...,{'error': 'Failed to fetch the page. Status co...,Error: Failed to fetch the page. Status code: 404


In [ ]:
url_df[url_df['text'].str.len() < 500]

,URL_ID,URL,content,text
24,11668.0,https://insights.blackcoffer.com/how-neural-ne...,{'error': 'Failed to fetch the page. Status co...,Error: Failed to fetch the page. Status code: 404
37,17671.4,https://insights.blackcoffer.com/covid-19-envi...,{'error': 'Failed to fetch the page. Status co...,Error: Failed to fetch the page. Status code: 404


In [ ]:
url_df.head()

,URL_ID,URL,content,text
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,{'title': 'Rise of e-health and its impact on ...,Rise of e-health and its impact on humans by t...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,{'title': 'Rise of e-health and its impact on ...,Rise of e-health and its impact on humans by t...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...


In [ ]:
def import_text_from_drive_folder(folder_path, encoding='utf-8'):
    """
    Imports text from all text files present in a Google Drive folder into a list.

    Args:
    folder_path (str): The path to the Google Drive folder containing text files.
    encoding (str, optional): The encoding to use when reading the text files. Default is 'utf-8'.

    Returns:
    list: A list of text contents from all text files.
    """
    # Mount Google Drive (if not already mounted)
    drive.mount('/content/drive')

    # Initialize an empty list to store text contents
    text_list = []

    # Traverse the directory and read text files
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):  # Check if the file has a .txt extension
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding=encoding, errors='replace') as f:
                    text = f.read()
                    text_list.append(text)

    return text_list

# Example usage with 'latin-1' encoding:
folder_path = '/content/drive/MyDrive/xyz/blackcoffer assignment/StopWords'  # Replace with the path to your Google Drive folder
text_contents = import_text_from_drive_folder(folder_path, encoding='latin-1')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
text_contents[0]

'UNITED  | Geographic\nSTATE\nNORTH\nSOUTH\nEAST\nNORTHEAST\nNORTHWEST\nSOUTHEAST\nSOUTHWEST\nWEST\nOCEAN\nSEA\nLAKE\nRIVER\nCREEK\nGULF\nMOUNTAIN\nSTREET\nBOULEVARD\nBLVD\nPARKWAY\nCITY\nCOUNTY\nCOUNTRY\nPACIFIC\nATLANTIC\nINDIAN\nMEDITERRANEAN\nCOMMONWEALTH\nAMERICA\nAMERICAN\nYORK  | Cities\nCHICAGO\nLAS\nVEGAS\nLOS\nANGELES\nMILWAUKEE\nSUNNYVALE\nFREMONT\nCINCINNATI\nPHILADELPHIA\nMIAMI\nDALLAS\nFORT\nBOSTON\nHOUSTON\nWASHINGTON\nATLANTA\nDETROIT\nSAN\nFRANSICO\nPHOENIX\nSEATTLE\nDIEGO\nMINNEAPOLIS\nMEMPHIS\nDENVER\nST\nLOUIS\nPITTSBURGH\nMANHATTAN\nHOLLYWOOD\nCOLUMBUS\nINDIANAPOLIS\nMUMBAI\nKARACHI\nONTARIO\nTORONTO\nCAMBRIDGE\nDELHI\nSAO\nPAULO\nSHANGHAI\nMOSCOW\nSEOUL\nISTANBUL\nTOKYO\nJAKARTA\nBEIJING\nLONDON\nLUXEMBOURG\nSINGAPORE\nREPUBLIC  | Countries\nCHINA\nINDIA\nINDONESIA\nBRAZIL\nBRAZILIAN\nPAKISTAN\nBANGLADESH\nRUSSIA\nNIGERIA\nNOVA\nSCOTIA\nJAPAN\nMALAYSIA\nMEXICO\nMEXICAN\nPHILIPPINES\nVIETNAM\nGERMANY\nFRANCE\nKOREA\nSPAIN\nARGENTINA\nBERMUDA\nCOLUMBIA\nPUERTO\nCANA

In [ ]:
text_contents[3].split('\n')

['ERNST',
 'YOUNG',
 'DELOITTE',
 'TOUCHE',
 'KPMG',
 'PRICEWATERHOUSECOOPERS',
 'PRICEWATERHOUSE',
 'COOPERS',
 '']

In [ ]:
# make a single list of stopwords from imported text
stop_words = []
for i in text_contents:
    stop_words += i.split('\n')

In [ ]:
len(stop_words)

14111

In [ ]:
# removing extra info in stop words
stop_words = [i.split()[0].lower() for i in stop_words if i != '']
len(stop_words)

14107

In [ ]:
# Function to remove stopwords from a text
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [ ]:
url_df['cleaned_text'] = url_df['text'].apply(remove_stopwords)

In [ ]:
import string
import re

# function to remove punctuations and extra spaces
def remove_punctuation(text):
    """
    Removes punctuation from the given text.

    Args:
    text (str): The input text.

    Returns:
    str: Text with punctuation removed.
    """
    # Define a translation table to remove punctuation
    translator = str.maketrans('', '', string.punctuation)

    # Use translate to remove punctuation
    text_without_punctuation = text.translate(translator)
    # remove extra white spaces
    cleaned_text = re.sub(' +', ' ', text_without_punctuation)

    return cleaned_text


In [ ]:
url_df['cleaned_text'] = url_df['cleaned_text'].apply(remove_punctuation)

In [ ]:
url_df.head()

,URL_ID,URL,content,text,cleaned_text
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood 2040 Black...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,{'title': 'Rise of e-health and its impact on ...,Rise of e-health and its impact on humans by t...,Rise ehealth impact humans 2030 Blackcoffer In...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,{'title': 'Rise of e-health and its impact on ...,Rise of e-health and its impact on humans by t...,Rise ehealth impact humans 2030 Blackcoffer In...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood 2040 Black...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood 2040 Black...


In [ ]:
def import_text(file_path, encoding='utf-8'):
    """
    Imports text from a file given its file path.

    Args:
    file_path (str): The path to the text file.
    encoding (str, optional): The encoding to use when reading the text file. Default is 'utf-8'.

    Returns:
    str: The text content from the file.
    """
    try:
        with open(file_path, 'r', encoding=encoding, errors='replace') as file:
            text = file.read()
        return text
    except FileNotFoundError:
        return f"File not found: {file_path}"
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:
# get list of negative words
file_path = '/content/drive/MyDrive/xyz/blackcoffer assignment/MasterDictionary/negative-words.txt'  # Replace with the actual file path
text_content = import_text(file_path, encoding='latin-1')  # Try 'latin-1' encoding

negative_words = text_content.split('\n')
# remove stop words
negative_words = [i.lower() for i in negative_words if i.lower() not in stop_words]

In [ ]:
# get list of positive words
file_path = '/content/drive/MyDrive/xyz/blackcoffer assignment/MasterDictionary/positive-words.txt'  # Replace with the actual file path
text_content = import_text(file_path, encoding='latin-1')  # Try 'latin-1' encoding

positive_words = text_content.split('\n')
# remove stop words
positive_words = [i.lower() for i in positive_words if i.lower() not in stop_words]

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')  # Download the required data

def tokenize_text(text, token_type='word'):
    """
    Tokenizes text into a list of tokens.

    Args:
    text (str): The input text.
    token_type (str, optional): The type of tokens to extract ('word' or 'sentence'). Default is 'word'.

    Returns:
    list: A list of tokens.
    """
    if token_type == 'word':
        # Tokenize into words
        tokens = word_tokenize(text)
    elif token_type == 'sentence':
        # Tokenize into sentences
        tokens = sent_tokenize(text)
    else:
        raise ValueError("Invalid token_type. Use 'word' or 'sentence'.")

    return tokens


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
url_df['tokens'] = url_df['cleaned_text'].apply(tokenize_text)

In [ ]:
url_df.head()

,URL_ID,URL,content,text,cleaned_text,tokens
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood 2040 Black...,"[Rise, telemedicine, Impact, Livelihood, 2040,..."
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,{'title': 'Rise of e-health and its impact on ...,Rise of e-health and its impact on humans by t...,Rise ehealth impact humans 2030 Blackcoffer In...,"[Rise, ehealth, impact, humans, 2030, Blackcof..."
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,{'title': 'Rise of e-health and its impact on ...,Rise of e-health and its impact on humans by t...,Rise ehealth impact humans 2030 Blackcoffer In...,"[Rise, ehealth, impact, humans, 2030, Blackcof..."
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood 2040 Black...,"[Rise, telemedicine, Impact, Livelihood, 2040,..."
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood 2040 Black...,"[Rise, telemedicine, Impact, Livelihood, 2040,..."


In [ ]:
# function to calculate positive score
def cal_positive_score(tokens_list):
  '''
  This score is calculated by assigning the value of +1 for each word if found in the Positive Dictionary and
  then adding up all the values.

  '''
  return sum([1 for i in tokens_list if i.lower() in positive_words])

In [ ]:
# function to calculate negative score
def cal_negative_score(tokens_list):
  '''
  This score is calculated by assigning the value of -1 for each word if found in the Negative Dictionary and
  then adding up all the values. We multiply the score with -1 so that the score is a positive number.
  '''
  return sum([1 for i in tokens_list if i.lower() in negative_words])

In [ ]:
url_df['POSITIVE SCORE'] = url_df['tokens'].apply(cal_positive_score)
url_df['NEGATIVE SCORE'] = url_df['tokens'].apply(cal_negative_score)

In [ ]:
url_df.head()

,URL_ID,URL,content,text,cleaned_text,tokens,POSITIVE SCORE,NEGATIVE SCORE
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood 2040 Black...,"[Rise, telemedicine, Impact, Livelihood, 2040,...",78,23
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,{'title': 'Rise of e-health and its impact on ...,Rise of e-health and its impact on humans by t...,Rise ehealth impact humans 2030 Blackcoffer In...,"[Rise, ehealth, impact, humans, 2030, Blackcof...",38,13
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,{'title': 'Rise of e-health and its impact on ...,Rise of e-health and its impact on humans by t...,Rise ehealth impact humans 2030 Blackcoffer In...,"[Rise, ehealth, impact, humans, 2030, Blackcof...",18,27
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood 2040 Black...,"[Rise, telemedicine, Impact, Livelihood, 2040,...",35,26
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood 2040 Black...,"[Rise, telemedicine, Impact, Livelihood, 2040,...",35,26


In [ ]:
url_df['POLARITY SCORE'] = (url_df['POSITIVE SCORE'] - url_df['NEGATIVE SCORE'])/ ((url_df['POSITIVE SCORE'] + url_df['NEGATIVE SCORE']) + 0.000001)

In [ ]:
url_df['SUBJECTIVITY SCORE'] = (url_df['POSITIVE SCORE'] + url_df['NEGATIVE SCORE'])/ (url_df['tokens'].apply(len) + 0.000001)

In [ ]:
url_df['AVG SENTENCE LENGTH'] = url_df['text'].str.split().apply(len) / url_df['text'].apply(lambda x: tokenize_text(x, token_type='sentence')).apply(len)

In [ ]:
url_df['AVG NUMBER OF WORDS PER SENTENCE'] = url_df['text'].str.split().apply(len) / url_df['text'].apply(lambda x: tokenize_text(x, token_type='sentence')).apply(len)

In [ ]:
url_df.describe()

,URL_ID,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,AVG NUMBER OF WORDS PER SENTENCE
count,114.00000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000
mean,26676.50000,29.745614,28.929825,0.072380,0.104012,22.868337,22.868337
std,15267.04828,17.873603,21.901113,0.444047,0.042802,15.101513,15.101513
min,123.00000,0.000000,0.000000,-1.000000,0.022281,4.500000,4.500000
25%,13630.65000,20.000000,11.000000,-0.294580,0.078036,17.680472,17.680472
50%,26676.50000,26.000000,25.000000,0.082768,0.095536,21.334328,21.334328
75%,39722.35000,38.000000,41.000000,0.401948,0.121425,24.290179,24.290179
max,52768.20000,85.000000,93.000000,1.000000,0.285714,165.363636,165.363636


In [ ]:
url_df.head()

,URL_ID,URL,content,text,cleaned_text,tokens,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,AVG NUMBER OF WORDS PER SENTENCE
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood 2040 Black...,"[Rise, telemedicine, Impact, Livelihood, 2040,...",78,23,0.544554,0.117716,20.691358,20.691358
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,{'title': 'Rise of e-health and its impact on ...,Rise of e-health and its impact on humans by t...,Rise ehealth impact humans 2030 Blackcoffer In...,"[Rise, ehealth, impact, humans, 2030, Blackcof...",38,13,0.490196,0.172881,24.115385,24.115385
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,{'title': 'Rise of e-health and its impact on ...,Rise of e-health and its impact on humans by t...,Rise ehealth impact humans 2030 Blackcoffer In...,"[Rise, ehealth, impact, humans, 2030, Blackcof...",18,27,-0.200000,0.083333,15.657143,15.657143
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood 2040 Black...,"[Rise, telemedicine, Impact, Livelihood, 2040,...",35,26,0.147541,0.089574,20.409836,20.409836
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,{'title': 'Rise of telemedicine and its Impact...,Rise of telemedicine and its Impact on Livelih...,Rise telemedicine Impact Livelihood 2040 Black...,"[Rise, telemedicine, Impact, Livelihood, 2040,...",35,26,0.147541,0.089574,20.409836,20.409836


In [ ]:
import nltk
from nltk.corpus import cmudict
nltk.download('punkt')
nltk.download('cmudict')

def count_complex_words(text):
    """
    Counts complex words in the given text.

    Args:
    text (str): The input text.

    Returns:
    int: The count of complex words.
    """
    # Create a CMU Pronouncing Dictionary
    cmu_dict = cmudict.dict()

    # Tokenize the text into words
    words = nltk.word_tokenize(text)

    # Initialize a count for complex words
    complex_word_count = 0

    # Function to count syllables in a word using CMU Pronouncing Dictionary
    def count_syllables(word):
        if word.lower() in cmu_dict:
            return max([len(list(y for y in x if y[-1].isdigit())) for x in cmu_dict[word.lower()]])
        else:
            return 0

    # Count complex words (words with more than two syllables)
    for word in words:
        syllable_count = count_syllables(word)
        if syllable_count > 2:
            complex_word_count += 1


    return complex_word_count



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [ ]:
# Complex Word Count
url_df['COMPLEX WORD COUNT'] = url_df['text'].apply(count_complex_words)

In [ ]:
# PERCENTAGE OF COMPLEX WORDS
url_df['PERCENTAGE OF COMPLEX WORDS'] = url_df['COMPLEX WORD COUNT'] / url_df['text'].str.split().apply(len)

In [ ]:
# FOG INDEX
url_df['FOG INDEX'] = 0.4 * (url_df['AVG SENTENCE LENGTH'] + url_df['PERCENTAGE OF COMPLEX WORDS'])

In [ ]:
from nltk.corpus import stopwords

nltk.download('stopwords')

def count_cleaned_words(text):
    """
    Counts the total cleaned words in the given text by removing stop words and punctuation.

    Args:
    text (str): The input text.

    Returns:
    int: The count of cleaned words.
    """
    # Tokenize the text into words
    words = nltk.word_tokenize(text)

    # Define a set of English stop words
    stop_words = set(stopwords.words('english'))

    # Define a set of punctuation characters
    punctuation = set(string.punctuation)

    # Function to check if a word is a stop word or contains punctuation
    def is_valid_word(word):
        return word.lower() not in stop_words and all(char not in punctuation for char in word)

    # Count cleaned words (non-stop words without punctuation)
    cleaned_word_count = sum(1 for word in words if is_valid_word(word))

    return cleaned_word_count


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Counts the total cleaned words in the given text by removing stop words and punctuation.
url_df['WORD COUNT'] = url_df['text'].apply(count_cleaned_words)

In [ ]:
from nltk.corpus import words

nltk.download('words')
nltk.download('punkt')

def count_syllables(word):
    """
    Count the number of syllables in a word while handling exceptions.

    Args:
    word (str): The input word.

    Returns:
    int: The count of syllables in the word.
    """
    # Check for exceptions: words ending with "es" and "ed" (e.g., "bakes", "jumped")
    if word.endswith(('es', 'ed')):
        return 0

    # Tokenize the word into phonemes using CMU Pronouncing Dictionary
    phonemes = nltk.word_tokenize(word.lower())

    # Count the vowels (phonemes ending in numbers) to estimate syllables
    syllable_count = sum(1 for phoneme in phonemes if phoneme[-1].isdigit())

    # Handle special case: words with no vowels
    if syllable_count == 0:
        return 1  # At least one syllable for words with no vowels

    return syllable_count

def count_syllables_in_text(text):
    """
    Count the number of syllables in each word of the text.

    Args:
    text (str): The input text.

    Returns:
    list: A list of syllable counts for each word in the text.
    """
    # Tokenize the text into words
    words = nltk.word_tokenize(text)

    # Count syllables for each word in the text
    syllable_counts = [count_syllables(word) for word in words]

    return syllable_counts


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# count the number of Syllables in each word of the text by counting the vowels present in each word.
# We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.
url_df['SYLLABLE PER WORD'] = url_df['text'].apply(count_syllables_in_text)

In [ ]:
def count_personal_pronouns(text):
    """
    Count the personal pronouns (I, we, my, ours, and us) in the text using regex.

    Args:
    text (str): The input text.

    Returns:
    int: The count of personal pronouns.
    """
    # Define a regex pattern to match personal pronouns
    pronoun_pattern = r'\b(I|we|my|ours|us)\b'

    # Use re.findall to find all matches of personal pronouns
    pronoun_matches = re.findall(pronoun_pattern, text, flags=re.IGNORECASE)

    # Filter out instances of the country name "US"
    filtered_matches = [match for match in pronoun_matches if match != 'US']

    # Count the remaining personal pronouns
    personal_pronoun_count = len(filtered_matches)

    return personal_pronoun_count


In [ ]:
# PERSONAL PRONOUNS
url_df['PERSONAL PRONOUNS'] = url_df['text'].apply(count_personal_pronouns)

In [ ]:
def calculate_average_word_length(text):
    """
    Calculate the average word length in the given text.

    Args:
    text (str): The input text.

    Returns:
    float: The average word length.
    """
    # Split the text into words
    words = text.split()

    # Calculate the total number of characters in all words
    total_characters = sum(len(word) for word in words)

    # Calculate the total number of words
    total_words = len(words)

    # Calculate the average word length
    if total_words > 0:
        average_word_length = total_characters / total_words
    else:
        average_word_length = 0

    return average_word_length


In [ ]:
# AVG WORD LENGTH
url_df['AVG WORD LENGTH'] = url_df['text'].apply(calculate_average_word_length)


In [ ]:
columns = ['POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH']
# Define a function to drop values from a row
def drop_values(row, columns = columns):
  for i in columns:
    # Replace values to drop with None (or any other desired value)
    row[i] = None if row['URL_ID'] in [11668.0, 17671.4] else row[i]
  return row


In [ ]:
url_df = url_df.apply(drop_values, axis = 1)

In [ ]:
# [11668.0, 17671.4] Error: Failed to fetch the page. Status code: 404
url_df.loc[(url_df['URL_ID'].isin([11668.0, 17671.4]))]

,URL_ID,URL,content,text,cleaned_text,tokens,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
24,11668.0,https://insights.blackcoffer.com/how-neural-ne...,{'error': 'Failed to fetch the page. Status co...,Error: Failed to fetch the page. Status code: 404,Error Failed fetch page Status code 404,"[Error, Failed, fetch, page, Status, code, 404]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
37,17671.4,https://insights.blackcoffer.com/covid-19-envi...,{'error': 'Failed to fetch the page. Status co...,Error: Failed to fetch the page. Status code: 404,Error Failed fetch page Status code 404,"[Error, Failed, fetch, page, Status, code, 404]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


In [ ]:
url_df.describe()

,URL_ID,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,WORD COUNT,PERSONAL PRONOUNS,AVG WORD LENGTH
count,114.00000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000
mean,26676.50000,30.276786,29.410714,0.091529,0.100767,23.196343,23.196343,209.250000,0.174343,9.348275,679.589286,7.232143,5.220025
std,15267.04828,17.578247,21.794517,0.423840,0.035490,15.032533,15.032533,99.711702,0.038335,6.011669,316.091946,8.728107,0.316874
min,123.00000,3.000000,0.000000,-0.727273,0.022281,12.820513,12.820513,24.000000,0.076663,5.176205,92.000000,0.000000,4.317568
25%,13630.65000,20.750000,12.500000,-0.283582,0.077508,17.877382,17.877382,129.750000,0.146461,7.223611,426.500000,2.000000,4.996407
50%,26676.50000,27.000000,25.500000,0.092624,0.095183,21.407692,21.407692,207.000000,0.172159,8.631121,643.000000,4.500000,5.213183
75%,39722.35000,38.000000,41.250000,0.407504,0.120303,24.310855,24.310855,272.750000,0.195371,9.797494,917.500000,9.250000,5.429112
max,52768.20000,85.000000,93.000000,1.000000,0.197674,165.363636,165.363636,433.000000,0.285145,66.203728,2144.000000,46.000000,5.965105


In [ ]:
output_df = url_df[['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH']]
output_df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,78.0,23.0,0.544554,0.117716,20.691358,0.223747,8.366042,20.691358,375.0,1001.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2.0,5.715990
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,38.0,13.0,0.490196,0.172881,24.115385,0.216906,9.732916,24.115385,136.0,332.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3.0,5.529506
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,18.0,27.0,-0.200000,0.083333,15.657143,0.201642,6.343514,15.657143,221.0,619.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3.0,5.411496
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,35.0,26.0,0.147541,0.089574,20.409836,0.217671,8.251003,20.409836,271.0,756.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",7.0,5.632129
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,35.0,26.0,0.147541,0.089574,20.409836,0.217671,8.251003,20.409836,271.0,756.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",7.0,5.632129


In [ ]:
# Export the output DataFrame to an Excel file
output_df.to_excel('blackcoffer text analysis.xlsx', index=False)